In [1]:
# Get raw data
with open('input/15.txt', 'r') as f:
    rawinput = f.read().strip()

In [ ]:
from intcode import Program  # From Day 9
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Part 1
class Droid(Program):
    dir_to_vec = {1: (-1,0), 2: (1,0), 3: (0,-1), 4: (0,1)}
    vec_to_dir = dict(zip(*[*zip(*dir_to_vec.items())][::-1]))
    
    def __init__(self, instr, verbose=False):
        super().__init__(instr, [1], verbose)
        self.pos = (0,0)
        self.pos_hist = []
        self.last_mv = 1
        self.layout = {self.pos: '.'}
        
    def do_output(self, addr, val):
        if self.verbose:  print(f"Ptr={self.ptr:2d}:  OUTPUT value {val[0]}")
        testpos = tuple([sum(i) 
                         for i in zip(self.pos, 
                                      self.dir_to_vec[abs(self.last_mv)])])
        if testpos not in self.layout:
            self.layout[testpos] = '#.O'[val[0]]
            if self.verbose:  print(f"DETECTED {self.layout[testpos]} at {testpos}")
        if val[0]:
            if self.last_mv > 0:
                self.pos_hist += [self.pos]
            self.pos = testpos
            if self.verbose:  print(f"MOVED to {self.pos}")
            
        for d in [1,2,3,4]:
            if (z:=tuple([sum(i) for i in zip(self.pos, self.dir_to_vec[d])])) not in self.layout:
                self.input_val = [d]
                self.last_mv = self.input_val[0]
                if self.verbose:  print(f"TESTING {z} next")
                break
        else:
            if self.pos_hist:
                if self.verbose:  print(f"STEPPING BACK to {self.pos_hist[-1]}")
                self.input_val = [self.vec_to_dir[tuple([i-j for i,j in zip(self.pos_hist.pop(), self.pos)])]]
                self.last_mv = -self.input_val[0]
            else:
                self.instr[0] = 99
        
        self.ptr += 2
        
def step(nsteps, layout):
    return np.where((nsteps == -1) 
                    & (layout != '#')
                    & ((z:=np.max(np.stack(((z:=np.pad(nsteps, 1, constant_values=-1))[:-2, 1:-1],
                                            z[2:, 1:-1], z[1:-1, :-2], z[1:-1, 2:])),
                                  axis=0)) >= 0),
                    z+1,
                    nsteps)

droid = Droid([int(i) for i in rawinput.split(',')])
droid.do_exec()

py,px,pval = [np.array(i) for i in zip(*[[y,x,v] for (y,x),v in droid.layout.items()])]
y_adj, x_adj = np.min(py), np.min(px)
py -= np.min(y_adj)
px -= np.min(x_adj)

layout = np.full((np.max(py)+1, np.max(px)+1), '.')
layout[py,px] = pval

nsteps = -np.ones_like(layout, dtype=int)
nsteps[-y_adj, -x_adj] = 0
while np.any(nsteps != (nsteps:=step(nsteps, layout))):  pass

np.max(np.where(layout == 'O', nsteps, -1)).item()

208

In [4]:
# Step 2
nsteps = np.where(layout == 'O', 0, -1)
while np.any(nsteps != (nsteps:=step(nsteps, layout))):  pass
np.max(nsteps).item()

306